In [1]:
import csv
from os.path import join
from numpy import array, random, linalg, cross, multiply, where, column_stack, ones_like,zeros_like, sqrt, concatenate,mean
from sklearn.cluster import KMeans

In [2]:
def points_reader(file_path: str):
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for x,y,z in reader:
            yield float(x),float(y),float(z)

In [3]:
working_dir = R'D:\Programy\Documents\II_EiT\POI'

vertical_file_path = join(working_dir, 'vertical.xyz')
horizontal_file_path = join(working_dir, 'horizontal.xyz')
cylinder_file_path = join(working_dir, 'cylinder.xyz')
    
vertical_points = array([x for x in points_reader(vertical_file_path)])
horizontal_points = array([x for x in points_reader(horizontal_file_path)])
cylinder_points = array([x for x in points_reader(cylinder_file_path)])
all_clouds = concatenate([vertical_points, horizontal_points, cylinder_points])

In [4]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(all_clouds).predict(all_clouds)
points0 =[]
points1=[]
points2=[]
for i in range(len(kmeans)):
    if kmeans[i] == 0:
        points0.append(all_clouds[i])
    elif kmeans[i] == 1:
        points1.append(all_clouds[i])
    else:
        points2.append(all_clouds[i])
points0 = array(points0)
points1 = array(points1)
points2 = array(points2)

In [5]:
def ransac(all_points):
    t = 0.1
    d = 0
    best_points = []
    i=0
    while i<500:
        random_points = [all_points[x] for x in random.randint(len(all_points), size=3)] #3 losowe punkty
        #wektory na podstawie punktow
        vector1 = random_points[1] - random_points[0]
        vector2 = random_points[2] - random_points[0]
        #wektory normalne do wyznaczonych
        u1 = vector1/linalg.norm(vector1)
        u2 = vector2/linalg.norm(vector2)
        #rownanie plaszczyzny Wx*x+Wy*y+Wz*z+D=0
        W = cross(u1,u2) #wektor normalny do plaszczyzny
        D = -multiply(W, random_points[0]) #odsuniecie od poczatku ukladu wsp

        inliers =[]
        for x in range(len(all_points)):
            distance= ((((W[0]*all_points[x,0]+D[0])+(W[1]*all_points[x,1]+D[1])+(W[2]*all_points[x,2]+D[2])))/sqrt((W[0]**2)+(W[1]**2)+(W[2]**2)))
            if abs(distance)<t:
                inliers.append(all_points[x])

        model_size = len(inliers)
        if model_size > d:
            best_points = array(inliers)
        i = i+1
    
    # Dopasowanie plaszczyzny metodą najmniejszych kwadratów do najlepszych punktów
    A = column_stack((best_points[:,0],best_points[:,1] ,ones_like(best_points[:,0])))
    B = best_points[:,2]
    params = linalg.lstsq(A, B, rcond=None)[0]
    A,B,C = params
    D_values = - (A * best_points[:, 0] + B * best_points[:, 1] + C * best_points[:, 2])
    D = mean(D_values)
    
    return [A,B,C,D]

In [6]:
vector_points0 = ransac(points0)
vector_points1 = ransac(points1)
vector_points2 = ransac(points2)

C:\Users\olgak\AppData\Local\Temp\ipykernel_8472\2511174905.py:13: RuntimeWarning: invalid value encountered in divide
  u2 = vector2/linalg.norm(vector2)
C:\Users\olgak\AppData\Local\Temp\ipykernel_8472\2511174905.py:20: RuntimeWarning: invalid value encountered in scalar divide
  distance= ((((W[0]*all_points[x,0]+D[0])+(W[1]*all_points[x,1]+D[1])+(W[2]*all_points[x,2]+D[2])))/sqrt((W[0]**2)+(W[1]**2)+(W[2]**2)))
C:\Users\olgak\AppData\Local\Temp\ipykernel_8472\2511174905.py:12: RuntimeWarning: invalid value encountered in divide
  u1 = vector1/linalg.norm(vector1)


In [7]:
def distance_point_plane(abcd,xyz):

    numerator = abs(abcd[0] * xyz[0] + abcd[1] * xyz[1] + abcd[2] * xyz[2] + abcd[3])
    denominator = sqrt(abcd[0]**2+abcd[1]**2+abcd[2]**2)
    return numerator / denominator

distance0=[]
distance1=[]
distance2=[]
for i in range(len(points0)):
    distance0.append(distance_point_plane(vector_points0, points0[i]))
for i in range(len(points1)):
    distance1.append(distance_point_plane(vector_points1, points1[i]))
for i in range(len(points2)):
    distance2.append(distance_point_plane(vector_points2, points2[i]))
distance0 = mean(distance0)
distance1 = mean(distance1)
distance2 = mean(distance2)

In [8]:
print("parametry wektora 0:",vector_points0[:3])
print("parametry wektora 1:",vector_points1[:3])
print("parametry wektora 2:",vector_points2[:3])

if distance0>distance1 and distance0>distance2:
    print("Chmura 0 nie jest płaszczyzną")
    if vector_points1[2]>vector_points1[0] and vector_points1[2]>vector_points1[1]:
        print("Płaszczyzna 1 jest pozioma")
    else:
        print("Płaszczyzna 1 jest pionowa")
    if vector_points2[2]>vector_points2[0] and vector_points2[2]>vector_points2[1]:
        print("Płaszczyzna 2 jest pozioma")
    else:
        print("Płaszczyzna 2 jest pionowa")
if distance1>distance0 and distance1>distance2:
    print("Chmura 1 nie jest płaszczyzną")
    if vector_points0[2]>vector_points0[0] and vector_points0[2]>vector_points0[1]:
        print("Płaszczyzna 0 jest pozioma")
    else:
        print("Płaszczyzna 0 jest pionowa")
    if vector_points2[2]>vector_points2[0] and vector_points2[2]>vector_points2[1]:
        print("Płaszczyzna 2 jest pozioma")
    else:
        print("Płaszczyzna 2 jest pionowa")
if distance2>distance1 and distance2>distance0:
    print("Chmura 2 nie jest płaszczyzną")
    if vector_points0[2]>vector_points0[0] and vector_points0[2]>vector_points0[1]:
        print("Płaszczyzna 0 jest pozioma")
    else:
        print("Płaszczyzna 0 jest pionowa")
    if vector_points1[2]>vector_points1[0] and vector_points1[2]>vector_points1[1]:
        print("Płaszczyzna 1 jest pozioma")
    else:
        print("Płaszczyzna 1 jest pionowa")

parametry wektora 0: [-2.4685073877585153e-05, 0.00024905278409662025, -0.036696386263785]
parametry wektora 1: [-0.049526092439921225, 22.682519970551493, 242.74718383467382]
parametry wektora 2: [-30.430472309443203, 4.606502646100399, -547.6508712109597]
Chmura 2 nie jest płaszczyzną
Płaszczyzna 0 jest pionowa
Płaszczyzna 1 jest pozioma
